In [1]:
import torch


In [2]:
print(torch.__version__)

1.7.1


In [3]:
import math
from typing import Tuple

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

In [ ]:
class TransformerModel(nn.Module):
    
    def __init__(self, ntokin: int,
                d_model: int,
                nhead: int,
                d_hid: int,
                nlayers: int,
                dropout: float=0.5):
        super().__init__()
        self.model_type = "Trnasformer"
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)
        
        self.init_weights()
        
    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.weight.data.univorm_(_initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(_initrange, initrange)
        
        
    def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [ seq_len, seq_len]
            
        returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.self.decoder(output)
        return 
    
    def generate_squre_subsequent_mask(sz: init) -> Tensor:
        
        return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)
        
            

In [4]:
sz=3
print(torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1))

tensor([[0., -inf, -inf],
        [0., 0., -inf],
        [0., 0., 0.]])
